In [3]:
import numpy as np
import json

In [5]:
#READ IMAGE


# READ FILE
with open('../DATASET_Juggling/test_images/girls.json') as f:
  data = json.load(f)

In [15]:
# JSON PRETTY PRINT 
print(json.dumps(data, indent = 4))

{
    "version": 1.3,
    "people": [
        {
            "person_id": [
                -1
            ],
            "pose_keypoints_2d": [
                354.433,
                136.398,
                0.79646,
                354.445,
                178.388,
                0.910756,
                329.817,
                186.6,
                0.818131,
                300.642,
                222.16,
                0.891634,
                339.828,
                227.645,
                0.874069,
                381.803,
                182.96,
                0.738916,
                410.988,
                225.833,
                0.854024,
                377.27,
                231.291,
                0.815299,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
       

An array pose_keypoints_2d containing the body part locations and detection confidence formatted as x1,y1,c1,x2,y2,c2,.... The coordinates x and y are in the range [0, source size] while c is the confidence score in the range [0,1]. 

Therefore every three values represents a new body part. Based on the 15 keypoint map in range [0,14] from the `README.md` -> Right Wrist is keypoint 4 and Left Wrist is 7 

In [26]:
person      = 0
right_wrist = 4
left_wrist  = 7
keypoints   = data['people'][person]["pose_keypoints_2d"]
right_wrist_key = keypoints[3*right_wrist : 3*right_wrist +3]
left_wrist_key  = keypoints[3*left_wrist  : 3*left_wrist +3]

#Display

print('All Keypoints :', keypoints , '\n')
print('Right Wrist: '  , right_wrist_key)
print('Left Wrist : '  , left_wrist_key)

All Keypoints : [354.433, 136.398, 0.79646, 354.445, 178.388, 0.910756, 329.817, 186.6, 0.818131, 300.642, 222.16, 0.891634, 339.828, 227.645, 0.874069, 381.803, 182.96, 0.738916, 410.988, 225.833, 0.854024, 377.27, 231.291, 0.815299, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 360.849, 218.507, 0.802096] 

Right Wrist:  [339.828, 227.645, 0.874069]
Left Wrist :  [377.27, 231.291, 0.815299]


In [ ]:
print(data)